In [ ]:
%cd ..\src

In [ ]:
from os.path import isfile, join

import wandb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import numpy as np

from configs.utils import get_config, get_bool_from_config
from echovpr.datasets.utils import load_np_file, get_subset_dataset
from echovpr.models.utils import get_sparsity
from echovpr.models.single_esn import SingleESN
from echovpr.models.sparce_layer import SpaRCe
from echovpr.trainer.utils.simple_processor import process_dataset

import logging

logging.basicConfig(level=logging.INFO)
log = logging.getLogger()

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 250

In [ ]:
config = get_config("configs\\train_esn_oxford.ini", log=False)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
artifact_name = 'mscerri/echovpr_oxford_esn/esn_i32f1sez:v0'

In [ ]:
run = wandb.init()
artifact = run.use_artifact(artifact_name, type='model')
artifact_dir = artifact.download()

model_file = join(artifact_dir, 'model.pt')
esn_model_file = join(artifact_dir, 'esn_model.pt')

all_in_one = not isfile(esn_model_file)

In [ ]:
in_features=int(config['model_in_features'])
reservoir_size=int(config['model_reservoir_size'])
out_features=int(config['model_out_features'])

esn_alpha = float(config['model_esn_alpha'])
esn_gamma = float(config['model_esn_gamma'])
esn_rho = float(config['model_esn_rho'])
esn_num_connections = int(config['model_esn_num_connections'])
sparce_enabled = get_bool_from_config(config, 'model_sparce_enabled')

model = nn.ModuleDict()

esn_model = SingleESN(
  in_features, 
  reservoir_size, 
  alpha=esn_alpha, 
  gamma=esn_gamma, 
  rho=esn_rho,
  sparsity=get_sparsity(esn_num_connections, reservoir_size),
  device=device
)

if all_in_one:
  model["esn"] = esn_model

if sparce_enabled:
  model["sparce"] = SpaRCe(reservoir_size)

model["out"] = nn.Linear(in_features=reservoir_size, out_features=out_features, bias=True)

In [ ]:
if not all_in_one:
  esn_model.load_state_dict(torch.load(esn_model_file))

model.load_state_dict(torch.load(model_file))

In [ ]:
if not all_in_one:
  esn_model.eval().to(device)
  
model.eval().to(device)

In [ ]:
day_hidden_repr = torch.from_numpy(load_np_file(config['dataset_oxford_day_hidden_repr_file_path']))
night_hidden_repr = torch.from_numpy(load_np_file(config['dataset_oxford_night_hidden_repr_file_path']))

hr_day_dataset = day_hidden_repr
hr_night_dataset = night_hidden_repr

# Normalize dataset
max_n = hr_day_dataset.max()
_ = hr_day_dataset.divide_(max_n)
_ = hr_night_dataset.divide_(max_n)

In [ ]:
esn_day_repr  = process_dataset("Day", hr_day_dataset, esn_model, config, device, log)
esn_night_repr  = process_dataset("Night", hr_night_dataset, esn_model, config, device, log)

In [ ]:
# import matplotlib.pyplot as plt

# for i in range(reservoir_size):
#   plt.plot(esn_day_repr[:, i])

# plt.show()

In [ ]:
# for i in range(reservoir_size):
#   plt.plot(esn_night_repr[:, i])

# plt.show()